training mnist o use the model in soduku solver

In [1]:
import numpy as np 


In [2]:
data_train = np.loadtxt("data/mnist_train.csv", delimiter=",")
data_train.shape

(60000, 785)

In [3]:
data_test = np.loadtxt("data/mnist_test.csv", delimiter=",")

In [4]:
# delete zeros from train and test data
no_zeros_train = data_train[data_train[:, 0] != 0]
no_zeros_test = data_test[data_test[:, 0] != 0]


In [5]:
X_train = no_zeros_train[:, 1:]
y_train = no_zeros_train[:, 0]
X_test = no_zeros_test[:, 1:]
y_test = no_zeros_test[:, 0]

In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

for i in range(2, 21):
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(X_train, y_train)
    print(f"KNN with k={i}: {model.score(X_test, y_test)}")

KNN with k=2: 0.9624168514412417
KNN with k=3: 0.9700665188470067
KNN with k=4: 0.968070953436807
KNN with k=5: 0.969179600886918
KNN with k=6: 0.9681818181818181
KNN with k=7: 0.9697339246119734
KNN with k=8: 0.9672949002217295
KNN with k=9: 0.966629711751663
KNN with k=10: 0.9667405764966741
KNN with k=11: 0.9672949002217295
KNN with k=12: 0.9662971175166297
KNN with k=13: 0.9657427937915742
KNN with k=14: 0.9637472283813747
KNN with k=15: 0.9634146341463414
KNN with k=16: 0.9633037694013303
KNN with k=17: 0.9629711751662972
KNN with k=18: 0.9638580931263858
KNN with k=19: 0.9631929046563192
KNN with k=20: 0.9628603104212861


In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models

X_train_cnn = X_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
X_test_cnn = X_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0


# Define CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(X_train_cnn, y_train, epochs=5, validation_data=(X_test_cnn, y_test))


/home/parsamhb82/anaconda3/envs/tf_env/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-03-05 21:51:11.210518: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 169585472 exceeds 10% of free system memory.


Epoch 1/5
1690/1690 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8856 - loss: 0.3505

2025-03-05 21:51:35.423857: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{} for conv (f32[32,64,11,11]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,32,13,13]{3,2,1,0}, f32[64,32,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kRelu","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-03-05 21:51:35.633434: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{} for conv (f32[32,64,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,64,5,5]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv

1690/1690 ━━━━━━━━━━━━━━━━━━━━ 27s 14ms/step - accuracy: 0.8856 - loss: 0.3503 - val_accuracy: 0.9887 - val_loss: 0.0357
Epoch 2/5
1690/1690 ━━━━━━━━━━━━━━━━━━━━ 22s 13ms/step - accuracy: 0.9854 - loss: 0.0481 - val_accuracy: 0.9889 - val_loss: 0.0366
Epoch 3/5
1690/1690 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.9901 - loss: 0.0318 - val_accuracy: 0.9890 - val_loss: 0.0334
Epoch 4/5
1690/1690 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.9927 - loss: 0.0227 - val_accuracy: 0.9909 - val_loss: 0.0278
Epoch 5/5
1690/1690 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - accuracy: 0.9937 - loss: 0.0183 - val_accuracy: 0.9906 - val_loss: 0.0278


In [11]:
import pickle
with open('cnn_model.pkl', 'wb') as file:
    pickle.dump(model, file)

model.save('cnn_model.h5')

In [7]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))

0.9700665188470067


In [11]:
import pickle
with open('knnmodel.pkl', 'wb') as file:
    pickle.dump(model, file)